In [ ]:
import sympy as sp
import numpy as np
from IPython.display import display, Math, Latex
from sympy.utilities.lambdify import lambdify

# Loading variables

In [ ]:
m_L = sp.symbols(r"m_L")
m_M = sp.symbols(r"m_M")
w_L = sp.symbols(r"w_L")
I_L = sp.symbols(r"I_L")
h_L = sp.symbols(r"h_L")
r = sp.symbols(r"r")
I_M = sp.symbols(r"I_M")

In [ ]:
# Friction coefficients
mu = sp.symbols(r"\mu")
mu_S = sp.symbols(r"\mu_{S}")

# System gains
b_J = sp.symbols(r"b_J")
k_J = sp.symbols(r"k_J")

In [ ]:
# Positions
p_CN = sp.symbols(r"p_{CN}")
p_CT = sp.symbols(r"p_{CT}")
p_MN = sp.symbols(r"p_{MN}")
p_LN = sp.symbols(r"p_{LN}")
p_LT = sp.symbols(r"p_{LT}")
theta_L = sp.symbols(r"\theta_L")
d_T = sp.symbols(r"d_T")
d_N = sp.symbols(r"d_N")

# p_C - p_L
p_LCN = sp.symbols(r"p_{LCN}")
p_LCT = sp.symbols(r"p_{LCT}")

# p_C - p_M
p_MCN = sp.symbols(r"p_{MCN}")
p_MCT = sp.symbols(r"p_{MCT}")

# Velocities
v_MN = sp.symbols(r"v_{MN}")
v_MT = sp.symbols(r"v_{MT}")
v_LN = sp.symbols(r"v_{LN}")
v_LT = sp.symbols(r"v_{LT}")
d_theta_L = sp.symbols(r"\dot\theta_L")
d_theta_M = sp.symbols(r"\dot\theta_M")
d_d_T = sp.symbols(r"\dot{d}_T")
d_d_N = sp.symbols(r"\dot{d}_N")

In [ ]:
# Input forces
F_GT = sp.symbols(r"F_{GT}")
F_GN = sp.symbols(r"F_{GN}")
F_OT, F_ON, tau_O = sp.symbols(r"F_{OT}, F_{ON} \tau_O")

# Control inputs
dd_theta_Ld = sp.symbols(r"\ddot\theta_{Ld}")
dd_d_Nd = sp.symbols(r"\ddot{d}_{Nd}")
dd_d_Td = sp.symbols(r"\ddot{d}_{Td}")
dd_theta_Md = sp.symbols(r"\ddot\theta_{Md}")

outputs = [
    a_LT, a_LN, a_MT, a_MN, F_NM, F_FL, F_FM, F_NL, F_CN, F_CT, tau_M, dd_theta_L, dd_theta_M, dd_d_N, dd_d_T
] = sp.symbols(
    r"a_{LT}, a_{LN}, a_{MT}, a_{MN}, F_{NM}, F_{FL}, F_{FM}, F_{NL}, F_{CN}, F_{CT}, \tau_M, \ddot\theta_L, \ddot\theta_M, \ddot{d}_N, \ddot{d}_T"
)

outputs = list(outputs)

In [ ]:
a_LT

## Differentiation of $\vec d$

In [ ]:
t = sp.symbols("t")
theta_L_func = sp.Function(r'\theta_L')(t)
N_hat = sp.Function(r'\hat N')(theta_L_func)
T_hat = sp.Function(r'\hat T')(theta_L_func)

d_T_func = sp.Function(r"d_T")(t)
d_N_func = sp.Function(r"d_N")(t)
d_g = d_T_func*T_hat + d_N_func*N_hat

d_vel_g = sp.diff(d_g, t)

d_vel_g = d_vel_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_g = d_vel_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_g = sp.diff(d_vel_g, t)
d_acc_g = d_acc_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_g = d_acc_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_g = d_acc_g
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_g = d_acc_cos_g.subs(d_T_func, d_T)
d_acc_cos_g = d_acc_cos_g.subs(d_N_func, d_N)

In [ ]:
dd_d_g_T = d_acc_cos_g.subs(N_hat, 0).subs(T_hat, 1)
dd_d_g_T

In [ ]:
dd_d_g_N = d_acc_cos_g.subs(T_hat, 0).subs(N_hat, 1)
dd_d_g_N

In [ ]:
p_M_func = sp.Function(r"p_M")(t)
p_L_func = sp.Function(r"p_L")(t)
v_M = sp.symbols(r"v_M")
v_L = sp.symbols(r"v_L")
d_s = (p_M_func + r*N_hat) - (p_L_func + (w_L/2)*T_hat - (h_L/2)*N_hat)

d_vel_s = sp.diff(d_s, t)
d_vel_s = d_vel_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_s = d_vel_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_s = sp.diff(d_vel_s, t)
d_acc_s = d_acc_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_s = d_acc_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_s = d_acc_s
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_s = d_acc_cos_s.subs(d_T_func, d_T)
d_acc_cos_s = d_acc_cos_s.subs(d_N_func, d_N)

In [ ]:
dd_d_s_T = d_acc_cos_s.subs(N_hat, 0).subs(T_hat, 1)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_L_func, t, t), a_LT)
dd_d_s_T

In [ ]:
dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)

In [ ]:
dd_d_s_N = d_acc_cos_s.subs(T_hat, 0).subs(N_hat, 1)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_M_func, t, t), a_MN)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_L_func, t, t), a_LN)
dd_d_s_N

# Setting up equations and matrices

In [ ]:
nat_eqs = [
    # Link tangential force balance
    [m_L*a_LT, F_FL+F_GT+F_OT],
    # Link normal force balance
    [m_L*a_LN, F_NL + F_GN + F_ON, ],
    # Manipulator tangential force balance
    [m_M*a_MT, F_FM + F_CT, ],
    # Manipulator normal force balance
    [m_M*a_MN, F_NM+F_CN, ],
    # Link moment balance
    [I_L*dd_theta_L, (-w_L/2)*F_ON - (p_LCN) * \
     F_FL + (p_LCT)*F_NL + tau_O, ],
    # Manipulator moment balance
    [I_M*dd_theta_M, tau_M-F_FM*(p_MCN), ],
    # 3rd law normal forces
    [F_NL, -F_NM],
    # Friction relationship L
    [F_FL, mu*mu_S*F_NL],
    # Friction relationship M
    [F_FM, -F_FL],
    # d_T derivative is derivative
    [dd_d_s_T, dd_d_g_T],
    # d_N derivative is derivative
    [dd_d_s_N, dd_d_g_N],
    # dd_d_N = 0 in real life assuming we're in contact
    [dd_d_N, 0],
]

art_eqs = [
#     [dd_d_N, dd_d_Nd],
#     [dd_d_T, dd_d_Td],
#     [dd_theta_L, dd_theta_Ld],
#     [dd_theta_M, dd_theta_Md],
]

env_eqs = nat_eqs + art_eqs

In [ ]:
dd_d_s_N

In [ ]:
out_str = r"\begin{aligned}"
for eq in env_eqs:
    out_str += sp.latex(eq[0]) + r" &= " + sp.latex(eq[1]) + r" \\" + "\n"
out_str += "\end{aligned}"
display(Math(out_str))

In [ ]:
env_eqs[10][0]-a_MN

In [ ]:
env_eqs[7]

In [ ]:
eq_idxs = {3, 10, 1, 6, 7, 4}
out_str = r"\begin{aligned}"
for i, eq in enumerate(env_eqs):
    if i in eq_idxs:
        out_str += sp.latex(eq[0]) + r" &= " + sp.latex(eq[1]) + r" \\" + "\n"
out_str += "\end{aligned}"
display(Math(out_str))

In [ ]:
F_CN_exp = env_eqs[3][0] - (env_eqs[3][1] - F_CN) # Used eq  3
F_CN_exp = F_CN_exp.subs(a_MN, env_eqs[10][1] - (env_eqs[10][0] - a_MN)).expand() # Used eq 10
# F_CN_exp = F_CN_exp.collect(dd_theta_L)
F_CN_exp = F_CN_exp.subs(a_LN, env_eqs[1][1]/m_L) # Used eq 1
F_CN_exp = F_CN_exp.subs(F_NM, -F_NL).expand().collect(F_NL) # Use eq 6
F_CN_exp = F_CN_exp.subs(F_NL, F_FL/(mu*mu_S)) # Mu cannot be zero, used eq 7
F_FL_exp = (F_CN - (F_CN_exp - F_CN_exp.coeff(F_FL)*F_FL))/F_CN_exp.coeff(F_FL) # 1 + m_M/m_L cannot be zero
F_FL_exp

In [ ]:
dd_theta_exp = env_eqs[4][1]/I_L # Used eq 4
dd_theta_exp = dd_theta_exp.subs(I_L, (h_L**2+w_L**2)*m_L/12)
# dd_theta_exp = dd_theta_exp.expand()
dd_theta_exp = dd_theta_exp.subs(F_NL, F_FL/(mu*mu_S)) # Used eq 6 again
dd_theta_exp.subs(F_FL, F_FL_exp).expand().collect(dd_theta_L).collect(F_CN)

In [ ]:
A = []
b = []
for lhs, rhs in env_eqs:
    A_row = []
    b_term = rhs - lhs
    for output_term in outputs:
        try:
            coeff_L = lhs.coeff(output_term)
        except AttributeError:
            coeff_L = 0
        try:
            coeff_R = rhs.coeff(output_term)
        except AttributeError:
            coeff_R = 0
        coeff = coeff_L - coeff_R
        A_row.append(coeff)
        b_term += coeff * output_term
    A.append(A_row)
    b.append(b_term)
A = sp.SparseMatrix(A)
A.simplify()
b = sp.Matrix([b]).T
b.simplify()
x = sp.Matrix([outputs]).T
x.simplify()

In [ ]:
A_aug = A.row_join(b)
results = A_aug.rref()[0]
A_prime = results[:, :-1]
b_prime = results[:, -1]

In [ ]:
A_prime

# Dynamics
For each equation, we want $\alpha_\theta(t,\cdot)\ddot\theta_L+\alpha_x(t,\cdot)\ddot x + \beta(t,\cdot)u = \gamma(t,\cdot)$, where $x$ is the thing we're trying to control and $u$ is the control input.
## Simplify coefficients
### $\alpha/\beta$ terms

In [ ]:
p_MLN = sp.symbols("p_{MLN}")
p_MLN

In [ ]:
lhss = (A_prime@x)[-4:-1,:]
lhss

In [ ]:
term = lhss[0].coeff(dd_theta_L)
num, denom = sp.fraction(term)
denom = (denom/(2*m_L)).simplify()
num = (num/(2*m_L)).simplify()
num = num.collect(mu*mu_S*p_LCN).collect(p_LCT).collect(d_T*m_M+m_M*w_L/2).subs(d_T*m_M+m_M*w_L/2, m_M*p_LCT)
out_term = 0
for arg in sp.Add.make_args(num):
    out_term  += (arg/denom).simplify()
out_term0 = out_term.subs(I_L, (h_L**2+w_L**2)*m_L/12)
out_term0 = out_term0.collect((h_L**2+w_L**2)/(12*denom)).collect(12)
out_term0

In [ ]:
term = lhss[1].coeff(dd_theta_L)
num, denom = sp.fraction(term)
denom = (denom/(2*m_L)).simplify()
num = (num/(2*m_L)).simplify()
num = num.collect(mu*mu_S*p_LCN).collect(-p_LCT).collect((m_M*d_N-m_M*h_L/2-m_M*r))
num = num.subs(-m_M*h_L/2 - m_M*r + m_M*d_N, -m_M*p_MLN)
out_term = 0
for arg in sp.Add.make_args(num):
    out_term  += (arg/denom).simplify()
out_term1 = out_term
out_term1 = out_term1.subs(I_L, (h_L**2+w_L**2)*m_L/12).collect(mu_S*mu/(12*mu*mu_S*p_LCN - 12*p_LCT)).collect((h_L**2+w_L**2)).collect(12)

out_term1_0 = sp.Add.make_args(out_term1)[1]
num0, denom0 = sp.fraction(out_term1_0)
num0 /= mu*mu_S
denom0 /= mu*mu_S
denom0 = denom0.simplify()
out_term1 = out_term1 - out_term1_0
out_term1 += num0/denom0
out_term1 = out_term1.collect(12)
out_term1

In [ ]:
term = lhss[2].coeff(dd_theta_L)
term

### $\gamma/\beta$ terms

In [ ]:
rhss = b_prime[-4:-1,:]
rhss

In [ ]:
term = rhss[0]
num, denom = sp.fraction(term.simplify())
num = num.collect(mu*mu_S*p_LCN)
num = num.collect(p_LCT)
num = num.collect(num.coeff(p_LCT)).collect(d_theta_L**2)
num = num.subs(-m_L*m_M*h_L/2 - m_L*m_M*r + m_L*m_M*d_N, -m_L*m_M*p_MLN)
num = num.collect(w_L/2).collect(-F_ON).collect(-tau_O).collect(m_L).collect(m_M)
out_term = 0
for i, arg in enumerate(sp.Add.make_args(num)):
    arg_out = (arg/denom).simplify()
    if i == 1:
        arg_out = arg_out.expand()
        arg_out = arg_out.collect(m_M)
        arg_out = arg_out.collect(1/m_L)
    out_term += arg_out
out_term
out_term

In [ ]:
term = rhss[1]
num, denom = sp.fraction(term.simplify())
num /= m_L
num = num.expand()
denom /= m_L
num = num.collect(mu*mu_S*p_LCN).collect(-p_LCT).collect(num.coeff(mu*mu_S*p_LCN)).collect(mu*mu_S)
out_term = 0
for i, arg in enumerate(sp.Add.make_args(num)):
    arg_out = (arg/denom)
    out_term += arg_out
out_term
out_term.collect(tau_O).collect(F_ON).collect(w_L/2).collect(d_theta_L**2*m_M).subs(d_T + w_L/2, p_LCT).collect(m_M).collect(1+m_M/m_L)

In [ ]:
num

In [ ]:
denom

In [ ]:
denom

In [ ]:
sp.Add.make_args(num)[0]/denom

In [ ]:
denom

Actually... why do I want to formulate it like that? It only makes the problem harder.

Wait no. We benefit from having a positive definite term on the terms we care about.

In [ ]:
## Coeffs with positive terms on 

## Pack f

In [ ]:
f = A_prime[:,11].copy()
f

In [ ]:
A_prime@x

In [ ]:
(A_prime@x)[8].coeff(dd_theta_L)

In [ ]:
x[8]

In [ ]:
b_prime[8]

In [ ]:
b_prime

## Set up function

In [ ]:
def split_mu_terms(term):
    num = sp.fraction(term)[0]
    mu_num = num - num.coeff(mu*mu_S)*(mu*mu_S - 1)
    mu_num = mu_num.simplify()
    mu_term = mu_num/sp.fraction(term)[1]
    mu_term = mu_term.factor()
    mu_term /= mu_0
    mu_term = mu_term.simplify()
    
    no_mu_term = num.coeff(mu*mu_S).factor()/sp.fraction(term)[1].coeff(mu*mu_S)
    no_mu_term = no_mu_term.simplify()

    assert (term - mu_term*mu_0 - no_mu_term).simplify() == 0
    
    return mu_term, no_mu_term

## Setting our constant

In [ ]:
mu_0 = 1/(1-mu*mu_S)
mu_0

# $\ddot\theta_L$ dynamics

In [ ]:
theta_L_idx = 8
lhs = (A_prime@x)[theta_L_idx]
rhs = b_prime[theta_L_idx]

display(Math(sp.latex(lhs) + r" = " + sp.latex(rhs)))

## f and g term

In [ ]:
f_term = f[theta_L_idx].copy()

In [ ]:
term = f_term
num, denom = sp.fraction(term)
num /= 2
denom /= 2
f_mu = num.collect(mu*mu_S).coeff(mu*mu_S)
f = num - f_mu*mu*mu_S

f_mu = f_mu.factor()

f = f.simplify()
f = f.collect(2*I_L).collect(p_LCT*m_L*m_M)
f = f.subs(d_T+w_L/2, p_LCT)

In [ ]:
f_num = sp.fraction(f_term)[0]
interm = f_num.collect(m_L*m_M).collect(mu*mu_S*p_LCN).collect(p_LCT).collect(2*d_T+w_L)
interm.subs(2*d_T+w_L, p_LCT*2)

In [ ]:
f

In [ ]:
f_mu

In [ ]:
g_mu = denom.collect(mu*mu_S).coeff(mu*mu_S)
g = denom - g_mu*mu*mu_S

g

In [ ]:
g_mu

## alpha and beta

In [ ]:
b_term = b_prime[theta_L_idx].copy()

In [ ]:
term = b_term
num, denom = sp.fraction(term)
num /= 2
denom /= 2
alpha_mu = num.collect(mu*mu_S).coeff(mu*mu_S)
alpha = num - (num - num.coeff(mu*mu_S)*mu*mu_S).simplify()

alpha_mu = alpha_mu.simplify()
alpha_mu

# f = f.simplify()
# f = f.collect(2*I_L).collect(p_LCT*m_L*m_M)
# f = f.subs(d_T+w_L/2, p_LCT)

In [ ]:
denom

# toherm

In [ ]:
# f_mu, f_no_mu = split_mu_terms(f_term)

In [ ]:
f_mu

In [ ]:
str(f_mu)

In [ ]:
f_no_mu

In [ ]:
str(f_no_mu)

In [ ]:
(f_term - f_mu*mu_0 - f_no_mu).simplify()

## b term

In [ ]:
b_term = b_prime[theta_L_idx].copy()

In [ ]:
str(b_term)

In [ ]:
b_mu, b_no_mu = split_mu_terms(b_term)
# b_mu /= mu_0
# b_mu = b_mu.simplify()

In [ ]:
str(b_mu)

In [ ]:
str(b_no_mu)

In [ ]:
(b_term - b_mu*mu_0 - b_no_mu).simplify()

## Final dynamics

$F_{CN} + \left(\mu_0f_\mu(t)+f(t)\right)\ddot\theta_L = \mu_0b_\mu(t)+b(t)$

In [ ]:
new_lhs = F_CN + mu_0*f_mu*dd_theta_L + f_no_mu*dd_theta_L
new_rhs = b_mu*mu_0+b_no_mu

In [ ]:
(new_lhs - lhs).simplify()

In [ ]:
(new_rhs - rhs).simplify()

In [ ]:
display(Math(sp.latex(new_lhs) + r" = " + sp.latex(new_rhs)))